In [1]:
import sys
import numpy as np
np.set_printoptions(threshold=sys.maxsize)
import numpy.typing as npt
import cProfile

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 



In [6]:

class Biome():
    def __init__(self, grid: npt.NDArray[np.int8]):
        self.grid = grid
        self.val_grid = np.zeros(grid.shape, dtype=np.int8)

    def __repr__(self):
        return str(self.grid)
    
    def run(self, generations: int):
        for i in range(generations):
            self.grid = self._update(self.grid)
        
    def _update(self, grid: npt.NDArray[np.int8]) -> npt.NDArray[np.int8]:
        
        # get indices of all non-zero elements
        indices = np.argwhere(grid == 1)
   
        # get the surrounding indices
        surrounding_indices = np.array([[x + i, y + j] for x, y in indices for i in range(-1, 2) for j in range(-1, 2) if not (i == 0 and j == 0) ])
      
        # remove the indices that are out of bounds
        surrounding_indices = surrounding_indices[(surrounding_indices[:, 0] >= 0) 
                                                & (surrounding_indices[:, 0] < grid.shape[0])
                                                & (surrounding_indices[:, 1] >= 0) 
                                                & (surrounding_indices[:, 1] < grid.shape[1])]

        # get the unique surrounding indices
        each_indices = np.unique(surrounding_indices, axis=0)

        # set the surrounding indices to 1 if they have 3 neighbours
        grid[indices[:, 0], indices[:, 1]] = [1 if np.sum(np.all(surrounding_indices == i, axis=1)) == 2 else 0 for i in np.unique(indices, axis=0)]
        grid[each_indices[:, 0], each_indices[:, 1]] = [grid[i[0], i[1]] if np.sum(np.all(surrounding_indices == i, axis=1)) != 3 else 1 for i in each_indices  ]
       
        return grid
    
    def values(self, grid: npt.NDArray[np.int8]) -> npt.NDArray[np.int8]:
        # get indices of all non-zero elements
        indices = np.argwhere(grid == 1)
   
        # get the surrounding indices
        surrounding_indices = np.array([[x + i, y + j] for x, y in indices for i in range(-1, 2) for j in range(-1, 2) if not (i == 0 and j == 0) ])
      
        # remove the indices that are out of bounds
        surrounding_indices = surrounding_indices[(surrounding_indices[:, 0] >= 0) 
                                                & (surrounding_indices[:, 0] < grid.shape[0])
                                                & (surrounding_indices[:, 1] >= 0) 
                                                & (surrounding_indices[:, 1] < grid.shape[1])]
        
        val_grid = grid.copy()

        for i in np.unique(surrounding_indices, axis=0):
            val_grid[i[0], i[1]] = np.sum(np.all(surrounding_indices == i, axis=1))
        
        return val_grid



IndentationError: unexpected indent (3819509659.py, line 31)

In [7]:
seed = np.random.randint(0, 2, size=(10, 10), dtype=np.int8)

grid = np.zeros((100, 100), dtype=np.int8)
grid[50:60, 50:60] = seed

biome = Biome(grid)
cProfile.run('biome.run(100)')

         571426 function calls (571240 primitive calls) in 5.707 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       21    0.008    0.000    5.707    0.272 372490805.py:13(_update)
       21    0.050    0.002    0.050    0.002 372490805.py:19(<listcomp>)
       21    1.092    0.052    3.971    0.189 372490805.py:31(<listcomp>)
       20    0.461    0.023    1.569    0.078 372490805.py:32(<listcomp>)
        1    0.000    0.000    5.707    5.707 372490805.py:9(run)
    33374    0.038    0.000    3.490    0.000 <__array_function__ internals>:177(all)
       21    0.000    0.000    0.002    0.000 <__array_function__ internals>:177(argwhere)
       82    0.000    0.000    0.002    0.000 <__array_function__ internals>:177(moveaxis)
       21    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(ndim)
       21    0.000    0.000    0.001    0.000 <__array_function__ internals>:177(nonzero)
       41    0.000  

KeyboardInterrupt: 

In [ ]:
# 